In [1]:
#import dependecies
import pandas as pd
import time
import numpy as np
import re
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [2]:
#read file into a dataframe
db_df= pd.read_csv('diabetic_data.csv', sep=',')
db_df

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,MC,?,51,0,16,0,0,0,250.13,291,458,9,None,>8,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,MC,?,33,3,18,0,0,1,560,276,787,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,MC,?,53,0,9,1,0,0,38,590,296,13,None,None,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,MC,Surgery-General,45,2,21,0,0,1,996,285,998,9,None,None,No,No,No,No,No,No,Steady,No,No,Steady,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO


In [3]:
#Look at the information on the datframe (the data types and the number of rows)
db_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

# **ETL Process**

In [4]:
#drop weight, payor_code, medical speacialty
df_dropped_col_3=db_df.drop(columns=['weight','medical_specialty','payer_code'])

In [5]:
#replace question mark signs with Na
df_w_nulls=df_dropped_col_3.replace("?",np.nan)

In [6]:
#check if dataframe has been updated and can now contain null values.
df_w_nulls.isnull().any()

encounter_id                False
patient_nbr                 False
race                         True
gender                      False
age                         False
admission_type_id           False
discharge_disposition_id    False
admission_source_id         False
time_in_hospital            False
num_lab_procedures          False
num_procedures              False
num_medications             False
number_outpatient           False
number_emergency            False
number_inpatient            False
diag_1                       True
diag_2                       True
diag_3                       True
number_diagnoses            False
max_glu_serum               False
A1Cresult                   False
metformin                   False
repaglinide                 False
nateglinide                 False
chlorpropamide              False
glimepiride                 False
acetohexamide               False
glipizide                   False
glyburide                   False
tolbutamide   

In [7]:
#drop null values and reset the index
dropped_df=df_w_nulls.dropna(how='any',inplace=False).reset_index()
dropped_df=dropped_df.drop(columns=['index'])
dropped_df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
1,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
2,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
3,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
4,35754,82637451,Caucasian,Male,[50-60),2,1,2,3,31,6,16,0,0,0,414,411,250,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30


In [8]:
dropped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98053 entries, 0 to 98052
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   encounter_id              98053 non-null  int64 
 1   patient_nbr               98053 non-null  int64 
 2   race                      98053 non-null  object
 3   gender                    98053 non-null  object
 4   age                       98053 non-null  object
 5   admission_type_id         98053 non-null  int64 
 6   discharge_disposition_id  98053 non-null  int64 
 7   admission_source_id       98053 non-null  int64 
 8   time_in_hospital          98053 non-null  int64 
 9   num_lab_procedures        98053 non-null  int64 
 10  num_procedures            98053 non-null  int64 
 11  num_medications           98053 non-null  int64 
 12  number_outpatient         98053 non-null  int64 
 13  number_emergency          98053 non-null  int64 
 14  number_inpatient      

### Break down the Master dataset into normalized tables.
# Patient
*   patient_nbr (PK)
*   age
*   gender
*   race
# Admission_type
*   admission_type_id (PK)
*   admission_name
# Admission source
*   admission_source_id (PK)
*   asmission_source_name
# Discharge Table
-discharge_disposition_id (PK)
-discharge_name
# Medication Table
-medication__id (PK)
-medication_type
# Patient_medication Table
-medication__id (PK)
-encounter_id (PK)
-change_in_dosage
# Diagnosis Table
-diagnosis_id
-diagnosis_type
# Encounter Table
-encounter_id (PK)
-patient_nbr (FK)
-num_lab_procedures
-num_procedures
-num_medications
-number_emergency
-number_outpatient
-number_inpatient
-time_in_hospital
-readmitted
-admission_type_id (FK)
-admission_source_id (FK)
-discharge_disposition_id (FK)
-diag_1(FK)
-diag_2(FK)
-diag_3(FK)
-number_diagnoses
-change
-A1Cresult
-max_glu_serum
-diabetesMed







In [9]:
#create a patient table with unique patient number to remove duplicates
def patient_tab(df):
  uniq_patient_number=df.drop_duplicates(subset = "patient_nbr")
  patient_table=uniq_patient_number[['patient_nbr','age','race','gender']].reset_index().set_index('patient_nbr').drop(columns=['index'])
  return patient_table

In [10]:
#call on the patient tab fundtion to see the patient table
patient_table=patient_tab(dropped_df)
patient_table.head()

,age,race,gender
patient_nbr,,,
55629189,[10-20),Caucasian,Female
86047875,[20-30),AfricanAmerican,Female
82442376,[30-40),Caucasian,Male
42519267,[40-50),Caucasian,Male
82637451,[50-60),Caucasian,Male


In [11]:
patient_table.to_csv('Patient_Table.csv')

In [12]:
#read in the admission type csv file 
admission_type=pd.read_csv('admission_type.csv')
admission_type.head()

,admission_type_id,description
0,1,Emergency
1,2,Urgent
2,3,Elective
3,4,Newborn
4,5,Not Available


In [13]:
#read in the admission source csv file
admission_source=pd.read_csv('admission_source.csv')
admission_source.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   admission_source_id  25 non-null     int64 
 1   description          24 non-null     object
dtypes: int64(1), object(1)
memory usage: 528.0+ bytes


In [14]:
#read in the discharge csv file
discharge_disposition=pd.read_csv('discharge_disposition.csv')
discharge_disposition.head()

,discharge_disposition_id,description
0,1,Discharged to home
1,2,Discharged/transferred to another short term h...
2,3,Discharged/transferred to SNF
3,4,Discharged/transferred to ICF
4,5,Discharged/transferred to another type of inpa...


In [15]:
#Create Medication table
def medic_tab(df):
  medication_id=np.arange(1,24)
  medication_type=df.columns.values[21:44]
  Medication=pd.DataFrame({"medication_id":medication_id, "medication_type":medication_type})
  return(Medication)

In [16]:
#Preview the medication tab
Medication=medic_tab(dropped_df)
Medication.head(23)

,medication_id,medication_type
0,1,metformin
1,2,repaglinide
2,3,nateglinide
3,4,chlorpropamide
4,5,glimepiride
5,6,acetohexamide
6,7,glipizide
7,8,glyburide
8,9,tolbutamide
9,10,pioglitazone


In [17]:
Medication.to_csv('Medication Table.csv')

In [18]:
patient_med = pd.DataFrame()

In [19]:
#Create Patient medication table
def patient_medic(df):
#create two empty dataframes to update the new information with
  patient_med = pd.DataFrame()
#loop through medication type to get the corresponding columns
  for i in Medication['medication_type'].values:
    if i=='metformin':
  #retrieve the encounter id for all meedications that were changed
      patient_med['encounter_id']=df['encounter_id'].where((df[i]=='Down') | (df[i]=='Up') | (df[i]=='Steady')).dropna()
      #assign medical Id number for all medication types
      patient_med['medication_id'] = Medication['medication_id'].where(Medication['medication_type']==i).dropna()[0]
      #retrieve the change in dosage
      patient_med['change_in_dosage']=df[i].where((df[i]=='Up') | (df[i]=='Down') | (df[i]=='Steady')).dropna()
    else:
      patient_med_up = pd.DataFrame()
      patient_med_up['encounter_id']=df['encounter_id'].where((df[i]=='Down') | (df[i]=='Up') | (df[i]=='Steady')).dropna()
      patient_med_up['medication_id'] = Medication['medication_id'].where(Medication['medication_type']==i).dropna().values[0]
      patient_med_up['change_in_dosage']=df[i].where((df[i]=='Up') | (df[i]=='Down') | (df[i]=='Steady')).dropna()
      patient_med=patient_med.append(patient_med_up).dropna()
  return(patient_med)


In [34]:
#Preview patient medication table
patient_med=patient_medic(dropped_df)
patient_med=patient_med.set_index('encounter_id')
patient_med

,medication_id,change_in_dosage
encounter_id,,
55842.0,1.0,Steady
40926.0,1.0,Steady
221634.0,1.0,Steady
236316.0,1.0,Steady
252822.0,1.0,Steady
...,...,...
190396296.0,20.0,Steady
240136452.0,20.0,Steady
247067964.0,20.0,Steady


In [35]:
patient_med.to_csv('patient_medication.csv')

In [36]:
#read ICD code 9 files for diagnosis into a dataframe
diag_df=pd.read_csv('ICD-9_Code_1998.csv', sep=',')

In [37]:
#create the diagnosis table
#split columns into to two
# delimiter = '-'
# my_list = [x+delimiter for x in my_str.split(delimiter) if x]
diag_df_split=diag_df['ICD TITLE & CODE CROSS REFERENCE FILE'].str.split('\s-(\d\d\d+)|\s+-(E\d+)|\s-\*(\d+)', expand=True)
#drop empty columns and rename the columns
diag_df_new=diag_df_split.replace(np.nan, '', regex=True)
diag_df_new[1]=np.sum([diag_df_new[1].values, diag_df_new[2].values, diag_df_new[3].values, diag_df_new[4].values], axis=0)
diagnosis=diag_df_new.drop(columns=[2,3,4])
diagnosis_df=diagnosis.where(~(diagnosis[1].str.contains('-'))).dropna()
diagnosis_df=diagnosis_df.where(~(diagnosis[0].str.contains('-'))).dropna()
diagnosis_df=diagnosis_df.where(~(pd.isnull(diagnosis_df[1] )))
diagnosis_table=diagnosis_df[[1,0]].rename(columns={1:'diagnosis_code',0:'diagnosis_name'}).drop([5,6],axis=0).set_index('diagnosis_code')
diagnosis_table.head()

,diagnosis_name
diagnosis_code,
001,Cholera
001.0,Due to Vibrio cholerae
001.1,Due to Vibrio cholerae el tor
001.9,Unspecified
002,Typhoid and paratyphoid fevers


In [38]:
#add the diagnosist dataframe to a csv file
diagnosis_table.to_csv('Diagnosis_table.csv')

In [39]:
#create an encounter Table
def encount(df):
  new_patient_table=np.delete(patient_table.columns.values, [0])
  values=[]
  for i in df.columns.values:
    if (i in Medication.medication_type.values) or (i in new_patient_table):
      values.append(i)
  Encounter=dropped_df.drop(columns=values)
  return (Encounter)

In [40]:
#preview encounter table
Encounter = encount(dropped_df)
Encounter.to_csv('Encounter table.csv')
Encounter.head()

,encounter_id,patient_nbr,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,change,diabetesMed,readmitted
0,149190,55629189,[10-20),1,1,7,3,59,0,18,0,0,0,276,250.01,255,9,None,None,Ch,Yes,>30
1,64410,86047875,[20-30),1,1,7,2,11,5,13,2,0,1,648,250,V27,6,None,None,No,Yes,NO
2,500364,82442376,[30-40),1,1,7,2,44,1,16,0,0,0,8,250.43,403,7,None,None,Ch,Yes,NO
3,16680,42519267,[40-50),1,1,7,1,51,0,8,0,0,0,197,157,250,5,None,None,Ch,Yes,NO
4,35754,82637451,[50-60),2,1,2,3,31,6,16,0,0,0,414,411,250,9,None,None,No,Yes,>30


# **Quality assurance process**

#Assertion test 1

In [41]:
split_test=diag_df['ICD TITLE & CODE CROSS REFERENCE FILE'].str.split('\s-(\d\d\d+)|\s+-(E\d+)', expand=True)

In [42]:
split_test=split_test.replace(np.nan, '', regex=True)

In [43]:
all_columns = np.sum([split_test[0].values, ' -',split_test[1].values, split_test[2].values, split_test[3].values], axis=0)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [44]:
for i in range(len(all_columns)):
  if all_columns[i][-1] == '-':
    all_columns[i] = all_columns[i][0:-2]
all_columns

array(['Cause-of-death statistics such as those shown in the following table are classified according to the',
       'Ninth Revision of the International Classification of Diseases -ICD-9. The table shows',
       'mortality statistics by ICD-9 codes at the most detailed listing of causes of death. The ICD Title &',
       ..., ' Unspecified -E997.9',
       'Injury due to war operations but occurring after cessation of hostilities -E998',
       'Late effects of injury due to war operations -E999'], dtype=object)

In [45]:
def assert_diagnosis():
  assert np.array_equal(diag_df['ICD TITLE & CODE CROSS REFERENCE FILE'].values, all_columns)

#Assertion test 2

In [ ]:
#assert length of master dataframe columns
def len_patient_table(df):
  len(df.columns)==50

In [ ]:
def assert_patient_table_pass():
  assert len_patient_table(dropped_df)==50

In [ ]:
def assert_patient_table_1():
  assert len_patient_table(dropped_df)==40

# Assertion test 3

In [46]:
#assert the medications are correct
def check_meds(df):
  assert np.array_equal(Medication.medication_type.values, df.columns.values[24:47])

In [47]:
check_meds(db_df)

# Assertion test 4

In [48]:
#assert that that all the medications that had a change in dosage are added to the new table
def assert_patient_medication(df):
  assert df.iloc[:,21:44].where(df.iloc[:,21:44] != 'No').count().sum()==patient_med['medication_id'].count()

In [49]:
assert_patient_medication(dropped_df)